# Test set

In [ ]:
! pip install -U sentence-transformers
! pip install -q wandb py_vncorenlp

import py_vncorenlp
py_vncorenlp.download_model(save_dir='/kaggle/working/')
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/default-java'
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/kaggle/working')

In [2]:
import pandas as pd
import numpy as np
import torch
import pickle
import math
import json
from datasets import load_dataset, DatasetDict, Dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, models, losses, util, datasets
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from sentence_transformers.cross_encoder.evaluation import CESoftmaxAccuracyEvaluator
from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)
import numpy as np

2024-06-23 08:59:02.784332: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 08:59:02.784438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 08:59:02.887965: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
with open('/kaggle/input/law-train-evalute/chunk_id_mapping', 'rb') as f:
    idx_mapping = pickle.load(f)
with open('/kaggle/input/law-train-evalute/chunk_corpus', 'rb') as f:
    segmented_chunk_corpus = pickle.load(f)
with open('/kaggle/input/law-train-evalute/queries_relevant_chunks', 'rb') as f:
    segmented_queries_relevant_chunk = pickle.load(f)
    
    
with open('/kaggle/input/law-qa/query_set_evaluate.json', 'r') as file:
    query_test_set = [json.loads(line) for line in file]

In [4]:
def isArticle(text):
    count = text.count('_')
    if count == 1:
        return True
    elif count == 2:
        return False
     
    return False

In [17]:
query_test_relevant_chunks = []
for query in query_test_set:
    add_record = True
    relevant_chunks = []
    
    for relevant_doc in query['relevant_docs']:
        try:
            if isArticle(relevant_doc):
                relevant_chunks += idx_mapping[relevant_doc]
            else:
                relevant_chunks.append(relevant_doc)
#             relevant_chunks += idx_mapping[relevant_doc]
        except KeyError:
            add_record = False
#             continue
            
    if add_record:
        query_test_relevant_chunks.append({
            'query_id' : query['query_id'],
            'query' : query['query'],
            'relevant_docs' : query['relevant_docs'],
            'relevant_chunks' : relevant_chunks
        })

In [18]:
segmented_query_test_relevant_chunks = []

for query in query_test_relevant_chunks:
    segmented_query_test_relevant_chunks.append({
        'query_id' : query['query_id'],
        'query' : ''.join(rdrsegmenter.word_segment(query['query'])),
        'relevant_docs' : query['relevant_docs'],
        'relevant_chunks' : query['relevant_chunks']
    })

In [19]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = {}
queries = {}
relevant_docs = {}
corpus_idx = 0
query_idx = 0

for query in segmented_query_test_relevant_chunks:
    queries[query['query_id']] = query['query']
    relevant_docs[query['query_id']] = set(query['relevant_chunks'])

for doc in segmented_chunk_corpus:
    corpus[doc['chunk_id']] = doc['text']

# Vietnamese S-BERT

In [8]:
word_embedding_model = models.Transformer("keepitreal/vietnamese-sbert", max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode = 'mean')

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [9]:
IR_evaluator = InformationRetrievalEvaluator(queries,corpus,relevant_docs)
model.evaluate(IR_evaluator, output_path="/kaggle/working/finetuned_vietsbert_notrain_chunk")

{'cosine_accuracy@1': 0.3334164174144234,
 'cosine_accuracy@3': 0.5271684945164506,
 'cosine_accuracy@5': 0.6074277168494516,
 'cosine_accuracy@10': 0.7007311399135926,
 'cosine_precision@1': 0.3334164174144234,
 'cosine_precision@3': 0.24700897308075773,
 'cosine_precision@5': 0.19810568295114656,
 'cosine_precision@10': 0.135493519441675,
 'cosine_recall@1': 0.11147676048910934,
 'cosine_recall@3': 0.23838532439512974,
 'cosine_recall@5': 0.3112477522554658,
 'cosine_recall@10': 0.4127129082661542,
 'cosine_ndcg@10': 0.34902095610989803,
 'cosine_mrr@10': 0.4496510138895451,
 'cosine_map@100': 0.28518872570703785,
 'dot_accuracy@1': 0.248670654702559,
 'dot_accuracy@3': 0.42821535393818544,
 'dot_accuracy@5': 0.5060651379195746,
 'dot_accuracy@10': 0.614406779661017,
 'dot_precision@1': 0.248670654702559,
 'dot_precision@3': 0.19128724936302202,
 'dot_precision@5': 0.1550016616816218,
 'dot_precision@10': 0.10908109006314391,
 'dot_recall@1': 0.07993282234414484,
 'dot_recall@3': 0.1

In [10]:
df_vietsbert = pd.read_csv('/kaggle/working/finetuned_vietsbert_notrain_chunk/Information-Retrieval_evaluation_results.csv')
df_vietsbert

,epoch,steps,cosine-Accuracy@1,cosine-Accuracy@3,cosine-Accuracy@5,cosine-Accuracy@10,cosine-Precision@1,cosine-Recall@1,cosine-Precision@3,cosine-Recall@3,...,dot-Recall@1,dot-Precision@3,dot-Recall@3,dot-Precision@5,dot-Recall@5,dot-Precision@10,dot-Recall@10,dot-MRR@10,dot-NDCG@10,dot-MAP@100
0,-1,-1,0.333416,0.527168,0.607428,0.700731,0.333416,0.111477,0.247009,0.238385,...,0.079933,0.191287,0.179385,0.155002,0.237661,0.109081,0.326456,0.359564,0.270039,0.214181


## BK-AI bi-encoder

In [20]:
word_embedding_model = models.Transformer("bkai-foundation-models/vietnamese-bi-encoder", max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode = 'mean')

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [21]:
IR_evaluator = InformationRetrievalEvaluator(queries,corpus,relevant_docs)
model.evaluate(IR_evaluator, output_path="/kaggle/working/finetuned_bkai_notrain_chunk")

{'cosine_accuracy@1': 0.47662076098198247,
 'cosine_accuracy@3': 0.6693702030373959,
 'cosine_accuracy@5': 0.7276686081323825,
 'cosine_accuracy@10': 0.7926623482662892,
 'cosine_precision@1': 0.47662076098198247,
 'cosine_precision@3': 0.3369803857529076,
 'cosine_precision@5': 0.26265309455119484,
 'cosine_precision@10': 0.1655870665723151,
 'cosine_recall@1': 0.24257644943999637,
 'cosine_recall@3': 0.44612390903692645,
 'cosine_recall@5': 0.5390620509369767,
 'cosine_recall@10': 0.6387949290517383,
 'cosine_ndcg@10': 0.5414078490213767,
 'cosine_mrr@10': 0.5857758043431175,
 'cosine_map@100': 0.48004966786137493,
 'dot_accuracy@1': 0.45288770344564805,
 'dot_accuracy@3': 0.6517881443579555,
 'dot_accuracy@5': 0.7159109466006205,
 'dot_accuracy@10': 0.7815578901529585,
 'dot_precision@1': 0.45288770344564805,
 'dot_precision@3': 0.3249324115907318,
 'dot_precision@5': 0.2541179032170269,
 'dot_precision@10': 0.16158619563442383,
 'dot_recall@1': 0.2287127238855097,
 'dot_recall@3': 

In [22]:
df_bkai = pd.read_csv('/kaggle/working/finetuned_bkai_notrain_chunk/Information-Retrieval_evaluation_results.csv')
df_bkai

,epoch,steps,cosine-Accuracy@1,cosine-Accuracy@3,cosine-Accuracy@5,cosine-Accuracy@10,cosine-Precision@1,cosine-Recall@1,cosine-Precision@3,cosine-Recall@3,...,dot-Recall@1,dot-Precision@3,dot-Recall@3,dot-Precision@5,dot-Recall@5,dot-Precision@10,dot-Recall@10,dot-MRR@10,dot-NDCG@10,dot-MAP@100
0,-1,-1,0.525091,0.717597,0.772183,0.833832,0.525091,0.179661,0.410962,0.395419,...,0.171667,0.397391,0.382343,0.320671,0.491488,0.208292,0.610259,0.615997,0.535438,0.468784
1,-1,-1,0.476621,0.669370,0.727669,0.792662,0.476621,0.242576,0.336980,0.446124,...,0.228713,0.324932,0.429786,0.254118,0.523096,0.161586,0.625824,0.566108,0.523772,0.461407


## SimCSE-VietNamese-phobert-base

In [14]:
word_embedding_model = models.Transformer("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base", max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode = 'mean')

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [15]:
IR_evaluator = InformationRetrievalEvaluator(queries,corpus,relevant_docs)
model.evaluate(IR_evaluator, output_path="/kaggle/working/finetuned_simcse_notrain_chunk")

{'cosine_accuracy@1': 0.31414091060152877,
 'cosine_accuracy@3': 0.5124626121635095,
 'cosine_accuracy@5': 0.5923894981721503,
 'cosine_accuracy@10': 0.6942505815885677,
 'cosine_precision@1': 0.31414091060152877,
 'cosine_precision@3': 0.2351279494848787,
 'cosine_precision@5': 0.18816882685277503,
 'cosine_precision@10': 0.13100697906281158,
 'cosine_recall@1': 0.10277935137892234,
 'cosine_recall@3': 0.2227771760462041,
 'cosine_recall@5': 0.29032374244775494,
 'cosine_recall@10': 0.3923074664982475,
 'cosine_ndcg@10': 0.3303019939060714,
 'cosine_mrr@10': 0.43309242246804325,
 'cosine_map@100': 0.26621693187598855,
 'dot_accuracy@1': 0.2727650382186773,
 'dot_accuracy@3': 0.46153207045530076,
 'dot_accuracy@5': 0.5435360584911931,
 'dot_accuracy@10': 0.6498005982053838,
 'dot_precision@1': 0.2727650382186773,
 'dot_precision@3': 0.2055500166168162,
 'dot_precision@5': 0.16753074111000332,
 'dot_precision@10': 0.11809571286141576,
 'dot_recall@1': 0.08836956338987216,
 'dot_recall@3

In [16]:
df_vietsbert = pd.read_csv('/kaggle/working/finetuned_simcse_notrain_chunk/Information-Retrieval_evaluation_results.csv')
df_vietsbert

,epoch,steps,cosine-Accuracy@1,cosine-Accuracy@3,cosine-Accuracy@5,cosine-Accuracy@10,cosine-Precision@1,cosine-Recall@1,cosine-Precision@3,cosine-Recall@3,...,dot-Recall@1,dot-Precision@3,dot-Recall@3,dot-Precision@5,dot-Recall@5,dot-Precision@10,dot-Recall@10,dot-MRR@10,dot-NDCG@10,dot-MAP@100
0,-1,-1,0.314141,0.512463,0.592389,0.694251,0.314141,0.102779,0.235128,0.222777,...,0.08837,0.20555,0.193636,0.167531,0.256485,0.118096,0.351443,0.388754,0.292442,0.232955
